In [219]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [221]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1VEpP7kLJjlLR9MyTgu2FyFnOJArh6U2b",
    'housing_data_ordy':"1jHl0STmNul4uPk9NeFrgtIuOnt8SqrgF" # adress to itaration 6 
}


housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
housing_data_ordy = pd.read_csv(gd_path(files_id['housing_data_ordy']), sep=";")
#df = housing_data

# Pre-Processing Pipe

In [220]:
housing_data_ordy = housing_data_ordy.iloc[:, 1:] #drop broken index

## Ordinary columns

This part was done by hands. Many Bothans died to bring us this information.

In [222]:
# Remove the ordinary(categorical lables) columns 
columns_to_remove = ['LandContour', 'LandSlope', 'OverallQual', 'OverallCond',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC']

housing_data_sml = housing_data.drop(columns=columns_to_remove).copy()

In [223]:
housing_data_sml

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,MSSubClass,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id,Alley,LotShape,Utilities,LotConfig,Neighborhood,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Electrical,GarageType,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,60,2003,2003,196.0,706,0,150,856,854,0,1710,1,0,2,1,1,8,2003.0,548,61,0,0,0,2,2008,1,NaN,Reg,AllPub,Inside,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,SBrkr,Attchd,Y,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,RL,Feedr,GasA,Pave,Y,CBlock,20,1976,1976,0.0,978,0,284,1262,0,0,1262,0,1,2,0,1,6,1976.0,460,0,0,0,0,5,2007,2,NaN,Reg,AllPub,FR2,Veenker,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,SBrkr,Attchd,Y,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,60,2001,2002,162.0,486,0,434,920,866,0,1786,1,0,2,1,1,6,2001.0,608,42,0,0,0,9,2008,3,NaN,IR1,AllPub,Inside,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,SBrkr,Attchd,Y,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,RL,Norm,GasA,Pave,Y,BrkTil,70,1915,1970,0.0,216,0,540,961,756,0,1717,1,0,1,0,1,7,1998.0,642,35,272,0,0,2,2006,4,NaN,IR1,AllPub,Corner,Crawfor,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,SBrkr,Detchd,Y,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,RL,Norm,GasA,Pave,Y,PConc,60,2000,2000,350.0,655,0,490,1145,1053,0,2198,1,0,2,1,1,9,2000.0,836,84,0,0,0,12,2008,5,NaN,IR1,AllPub,FR2,NoRidge,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,SBrkr,Attchd,Y,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,60,1999,2000,0.0,0,0,953,953,694,0,1647,0,0,2,1,1,7,1999.0,460,40,0,0,0,8,2007,1456,NaN,Reg,AllPub,Inside,Gilbert,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,SBrkr,Attchd,Y,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,RL,Norm,GasA,Pave,Y,CBlock,20,1978,1988,119.0,790,163,589,2073,0,0,2073,1,0,2,0,1,7,1978.0,500,0,0,0,0,2,2010,1457,NaN,Reg,AllPub,Inside,NWAmes,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,SBrkr,Attchd,Y,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,RL,Norm,GasA,Pave,Y,Stone,70,1941,2006,0.0,275,0,877,1188,1152,0,2340,0,0,2,0,1,9,1941.0,252,60,0,0,2500,5,2010,1458,NaN,Reg,AllPub,Inside,Crawfor,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,SBrkr,Attchd,Y,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,RL,Norm,GasA,Pave,Y,CBlock,20,1950,1996,0.0,49,1029,0,1078,0,0,1078,1,0,1,0,1,5,1950.0,240,0,112,0,0,4,2010,1459,NaN,Reg,AllPub,Inside,NAmes,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,FuseA,Attchd,Y,NaN,NaN,WD,Normal


In [224]:
# Add the orinary(numerical lables) columns

df =pd.concat([housing_data_sml,housing_data_ordy], axis=1)

In [225]:
df.select_dtypes(include="number")

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,MSSubClass,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id,Unnamed: 0,LandContour,LandSlope,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
0,8450,65.0,856,3,0,0,2,0,0,0,60,2003,2003,196.0,706,0,150,856,854,0,1710,1,0,2,1,1,8,2003.0,548,61,0,0,0,2,2008,1,0,3,2,6,4,3,2,3,2,0,6,1,4,3,7,0,2,2,2,0
1,9600,80.0,1262,3,1,0,2,298,0,0,20,1976,1976,0.0,978,0,284,1262,0,0,1262,0,1,2,0,1,6,1976.0,460,0,0,0,0,5,2007,2,1,3,2,5,7,2,2,3,2,3,5,1,4,2,7,2,2,2,2,0
2,11250,68.0,920,3,1,0,2,0,0,0,60,2001,2002,162.0,486,0,434,920,866,0,1786,1,0,2,1,1,6,2001.0,608,42,0,0,0,9,2008,3,2,3,2,6,4,3,2,3,2,1,6,1,4,3,7,2,2,2,2,0
3,9550,60.0,756,3,1,0,3,0,0,0,70,1915,1970,0.0,216,0,540,961,756,0,1717,1,0,1,0,1,7,1998.0,642,35,272,0,0,2,2006,4,3,3,2,6,4,2,2,2,3,0,5,1,3,3,7,3,1,2,2,0
4,14260,84.0,1145,4,1,0,3,192,0,0,60,2000,2000,350.0,655,0,490,1145,1053,0,2198,1,0,2,1,1,9,2000.0,836,84,0,0,0,12,2008,5,4,3,2,7,4,3,2,3,2,2,6,1,4,3,7,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,60,1999,2000,0.0,0,0,953,953,694,0,1647,0,0,2,1,1,7,1999.0,460,40,0,0,0,8,2007,1456,1455,3,2,5,4,2,2,3,2,0,1,1,4,2,7,2,2,2,2,0
1456,13175,85.0,1542,3,2,0,2,349,0,0,20,1978,1988,119.0,790,163,589,2073,0,0,2073,1,0,2,0,1,7,1978.0,500,0,0,0,0,2,2010,1457,1456,3,2,5,5,2,2,3,2,0,5,3,2,2,6,2,1,2,2,0
1457,9042,66.0,1152,4,2,0,1,0,0,1,70,1941,2006,0.0,275,0,877,1188,1152,0,2340,0,0,2,0,1,9,1941.0,252,60,0,0,2500,5,2010,1458,1457,3,2,6,8,4,3,2,3,0,6,1,4,3,7,3,2,2,2,0
1458,9717,68.0,1078,2,0,0,1,366,0,0,20,1950,1996,0.0,49,1029,0,1078,0,0,1078,1,0,1,0,1,5,1950.0,240,0,112,0,0,4,2010,1459,1458,3,2,4,5,2,2,2,2,1,6,3,3,3,7,0,1,2,2,0


In [226]:
from sklearn.model_selection import train_test_split

# X and y creation
y = df.pop("Expensive")

# Feature Engineering
X = df.drop(columns=["LotFrontage"])

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# building the pipeline
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))

categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(handle_unknown="ignore")
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categoric_pipe, X_cat.columns),
    ]
)

# Modelling Pipe - 1

## Decisiontree - Creation

In [227]:
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}

'''param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    'decisiontreeclassifier__max_depth': range(2, 20),
    'decisiontreeclassifier__min_samples_leaf': range(3, 15, 2),
    'decisiontreeclassifier__min_samples_split': range(3, 40, 5),
    'decisiontreeclassifier__criterion':['gini', 'entropy']
    }'''

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"dtree" : search.best_score_}

scores

Fitting 5 folds for each of 60 candidates, totalling 300 fits


{'dtree': 0.9289461134954697}

## Decisiontree - Analysis

### Confusion Matrix

In [228]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

Training Accuracy: 0.9537671232876712
Testing Accuracy: 0.9383561643835616
Confusion Matrix (Training):
 [[973  20]
 [ 34 141]]
Confusion Matrix (Testing):
 [[243   7]
 [ 11  31]]


### F-Score

In [229]:
from sklearn.metrics import f1_score
print("Training Accuracy:", f1_score(y_train,y_train_pred))
print("Testing Accuracy:", f1_score(y_test,y_test_pred))

Training Accuracy: 0.8392857142857142
Testing Accuracy: 0.775


### Kappa

In [230]:
from sklearn.metrics import cohen_kappa_score
print("Training Accuracy:", cohen_kappa_score(y_train,y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test,y_test_pred))

Training Accuracy: 0.8123404483216204
Testing Accuracy: 0.7393891312971044


In [250]:
#First Commission Export
# make predictions using the trained DecisionTreeClassifier model
predictions = search.best_estimator_.predict(X_test)

# convert predictions to a pandas dataframe
predictions_df = pd.DataFrame(predictions, columns=['Expensive'])

# concatenate the input dataframe with the predictions dataframe
result_df = pd.concat([X_test.Id, predictions_df], axis=1)

# save the concatenated dataframe as a CSV file
result_df.to_csv('commision2_anhe.csv', index=False)

In [252]:
X_test

,LotArea,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,MSSubClass,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id,Alley,LotShape,Utilities,LotConfig,Neighborhood,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Electrical,GarageType,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition,Unnamed: 0,LandContour,LandSlope,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
147,9505,884,3,1,0,2,144,0,RL,Norm,GasA,Pave,Y,PConc,60,2001,2001,180.0,0,0,884,884,1151,0,2035,0,0,2,1,1,8,2001.0,434,48,0,0,0,5,2010,148,NaN,IR1,AllPub,CulDSac,Gilbert,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,SBrkr,BuiltIn,Y,NaN,NaN,WD,Normal,147,3,2,6,4,3,2,3,2,0,1,1,4,3,7,3,3,2,2,0
676,9600,1095,4,0,0,3,0,0,RM,Norm,GasW,Pave,N,BrkTil,70,1900,1950,0.0,0,0,1095,1095,679,0,1774,1,0,2,0,2,8,1920.0,779,0,90,0,0,5,2006,677,Grvl,Reg,AllPub,Inside,OldTown,Norm,1Fam,2Story,Gable,CompShg,AsbShng,Stucco,None,SBrkr,2Types,N,NaN,NaN,WD,Normal,676,3,2,3,1,2,2,2,1,0,1,1,1,2,5,0,1,1,1,0
1304,3363,976,3,0,0,2,0,0,RM,Norm,GasA,Pave,Y,PConc,160,2004,2004,117.0,0,0,976,976,732,0,1708,0,0,2,0,1,7,2004.0,380,40,0,0,0,4,2006,1305,NaN,Reg,AllPub,Inside,Edwards,Norm,TwnhsE,2Story,Gable,CompShg,VinylSd,VinylSd,Stone,SBrkr,Detchd,Y,NaN,NaN,WD,Normal,1304,3,2,6,4,3,2,3,2,0,1,1,4,3,3,0,1,2,2,0
1372,9750,1108,3,1,0,2,253,0,RL,Norm,GasA,Pave,Y,PConc,60,1998,1998,0.0,975,0,133,1108,989,0,2097,1,0,2,1,1,8,1998.0,583,170,0,0,0,6,2006,1373,NaN,Reg,AllPub,Corner,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,SBrkr,Detchd,Y,NaN,NaN,WD,Normal,1372,3,2,6,5,2,2,3,2,2,6,1,4,3,7,2,2,2,2,0
1427,10930,913,3,1,0,1,0,0,RL,Artery,GasA,Pave,Y,CBlock,50,1945,1950,0.0,580,0,333,1048,510,0,1558,1,0,1,1,1,6,1962.0,288,0,0,0,0,4,2008,1428,Grvl,Reg,AllPub,Inside,NAmes,Norm,1Fam,1.5Fin,Gable,CompShg,MetalSd,MetalSd,None,FuseA,Attchd,Y,NaN,NaN,WD,Normal,1427,2,2,4,5,2,2,2,2,0,4,1,2,2,7,2,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,10592,602,3,2,0,1,96,53,RL,Norm,GasA,Pave,Y,PConc,70,1923,1996,0.0,0,0,602,900,602,0,1502,0,0,1,1,1,7,1923.0,180,0,112,0,0,8,2007,1013,NaN,Reg,AllPub,Inside,Crawfor,Norm,1Fam,2Story,Hip,CompShg,Wd Sdng,Wd Sdng,None,SBrkr,Detchd,Y,NaN,NaN,WD,Normal,1012,3,2,5,6,2,3,2,1,0,1,1,2,3,7,2,1,2,2,0
1059,11275,854,3,1,0,2,0,0,RL,Norm,GasA,Pave,Y,CBlock,50,1932,1950,480.0,297,557,0,1096,895,0,1991,0,0,1,1,1,7,1977.0,432,0,19,0,0,3,2007,1060,NaN,IR1,AllPub,Corner,Crawfor,Norm,1Fam,1.5Fin,Gable,CompShg,MetalSd,MetalSd,BrkFace,SBrkr,Detchd,Y,NaN,NaN,WD,Normal,1059,1,1,5,6,2,2,2,2,1,3,2,2,2,7,3,1,2,1,0
890,8064,672,3,1,0,2,0,0,RL,Artery,GasA,Pave,Y,CBlock,50,1949,2006,0.0,0,0,672,672,252,0,924,0,0,1,0,1,6,2003.0,576,0,0,0,2000,7,2007,891,NaN,Reg,AllPub,Corner,NAmes,Norm,1Fam,1.5Fin,Gable,CompShg,MetalSd,MetalSd,None,SBrkr,Detchd,Y,MnPrv,Shed,WD,Normal,890,3,2,4,6,2,3,2,2,1,1,1,4,2,7,0,1,2,2,0
628,11606,1040,5,2,0,2,335,0,RL,Norm,GasA,Pave,Y,PConc,60,1969,1969,192.0,650,0,390,1040,1040,0,2080,0,1,1,2,1,9,1969.0,504,0,0,0,0,9,2007,629,NaN,IR1,AllPub,Inside,NAmes,Norm,1Fam,2Story,Gable,CompShg,Plywood,Plywood,BrkFace,SBrkr,Attchd,Y,NaN,NaN,WD,Family,628,1,0,4,4,2,2,3,2,2,3,1,2,1,7,2,1,2,2,0


# Modeling Pipe - 2


In [ ]:
# Modeling Pipe - 2

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


full_pipeline = make_pipeline(preprocessor,
                              KNeighborsClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors": range(3, 10, 2)
}

search2 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search2.fit(X_train, y_train)

scores2 = {"knn" : search.best_score_}

scores2


Fitting 5 folds for each of 8 candidates, totalling 40 fits


{'knn': 0.9306591834488831}

## Decisiontree - Analysis

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

Training Accuracy: 0.9537671232876712
Testing Accuracy: 0.9383561643835616
Confusion Matrix (Training):
 [[973  20]
 [ 34 141]]
Confusion Matrix (Testing):
 [[243   7]
 [ 11  31]]


### F-Score

In [ ]:
from sklearn.metrics import f1_score
print("Training Accuracy:", f1_score(y_train,y_train_pred))
print("Testing Accuracy:", f1_score(y_test,y_test_pred))

Training Accuracy: 0.8392857142857142
Testing Accuracy: 0.775


### Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score
print("Training Accuracy:", cohen_kappa_score(y_train,y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test,y_test_pred))

Training Accuracy: 0.8123404483216204
Testing Accuracy: 0.7393891312971044
